# Introduction

In this micro-course, you will learn a practical approach to feature engineering. You'll be able to apply what you learn to Kaggle competitions as well as building machine learning applications. Through hands-on exercises you will:

* Develop a baseline model for comparing performance on models with more features
* Encode categorical features so the model can make better use of the information
* Generate new features to provide more information for the model
* Select features to reduce overfitting and increase prediction speed

In the exercise, you will apply feature engineering techniques to data from the TalkingData AdTracking competition.

<center><a href="https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection"><img src="https://i.imgur.com/srKxEkD.png" width=600px></a></center>

The goal of the competition is to predict if a user will download an app after clicking through an ad. For this course you will use a small sample of the data, dropping 99% of negative records (where the app wasn't downloaded) to make the target more balanced.

### Kickstarter projects

To help you apply the techniques, you'll first see them applied using data from Kickstarter projects. The first few rows of the Kickstarter projects data looks like this:

In [ ]:
import pandas as pd
ks = pd.read_csv('../input/kickstarter-projects/ks-projects-201801.csv',
                 parse_dates=['deadline', 'launched'])
ks.head(10)

What we can do here is predict if a Kickstarter project will succeed. We get the outcome from the `state` column. To predict the outcome we can use features such as category, currency, funding goal, country, and when it was launched. 

### Preparing target column

First I'll look at project states and convert the column into something we can use as targets in a model.

In [ ]:
pd.unique(ks.state)

We have six states, how many records of each?

In [ ]:
ks.groupby('state')['ID'].count()

Data cleaning isn't the current focus, so we'll simplify this example by:

- Dropping projects that are "live"
- Counting "successful" states as `outcome = 1`
- Combining every other state as `outcome = 0`

In [ ]:
# Drop live projects
ks = ks.query('state != "live"')

# Add outcome column, "successful" == 1, others are 0
ks = ks.assign(outcome=(ks['state'] == 'successful').astype(int))

### Converting timestamps

I convert the `launched` feature into categorical features we can use in a model. Since I loaded in the columns as timestamp data, I access date and time values through the `.dt` attribute on the timestamp column.

In [ ]:
ks = ks.assign(hour=ks.launched.dt.hour,
               day=ks.launched.dt.day,
               month=ks.launched.dt.month,
               year=ks.launched.dt.year)

ks.head()

### Prepping categorical variables

Now for the categorical variables -- `category`, `currency`, and `country` -- I'll need to convert them into integers so our model can use the data. For this I'll use scikit-learn's `LabelEncoder`. This assigns an integer to each value of the categorical feature and replaces those values with the integers.

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_features = ['category', 'currency', 'country']
encoder = LabelEncoder()

# Apply the label encoder to each column
encoded = ks[cat_features].apply(encoder.fit_transform)
encoded.head(10)

I'll collect all the features we'll use in a new dataframe and use that to train a model.

In [ ]:
# Since ks and encoded have the same index and I can easily join them
data = ks[['goal', 'hour', 'day', 'month', 'year', 'outcome']].join(encoded)
data.head()

### Creating training, validation, and test splits

We need to create data sets for training, validation, and testing. We'll use a fairly simple approach and split the data using slices. We'll use 10% of the data as a validation set, 10% for testing, and the other 80% for training.

In [ ]:
valid_fraction = 0.1
valid_size = int(len(data) * valid_fraction)

train = data[:-2 * valid_size]
valid = data[-2 * valid_size:-valid_size]
test = data[-valid_size:]

In general you want to be careful that each data set has the same proportion of target classes. I'll print out the fraction of successful outcomes for each of our datasets.

In [ ]:
for each in [train, valid, test]:
    print(f"Outcome fraction = {each.outcome.mean():.4f}")

This looks good, each set is around 35% true outcomes likely because the data was well randomized beforehand. A good way to do this automatically is with `sklearn.model_selection.StratifiedShuffleSplit` but I don't need to use it here.

### Training a LightGBM model

For this course we'll be using a LightGBM model. This is a tree-based model that typically provides the best performance, even compared to XGBoost. It's also relatively fast to train. We won't do hyperparameter optimization because that isn't the goal of this course. So, our models won't be the absolute best performance you can get. But you'll still see model performance improve as we do feature engineering.

In [ ]:
import lightgbm as lgb

feature_cols = train.columns.drop('outcome')

dtrain = lgb.Dataset(train[feature_cols], label=train['outcome'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['outcome'])

param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10, verbose_eval=False)

### Making predictions & evaluating the model

Finally, let's make predictions on the test set with the model and see how well it performs. An important thing to remember is that you can overfit to the validation data. This is why we need a test set that the model never sees until the final evaluation.

In [ ]:
from sklearn import metrics
ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['outcome'], ypred)

print(f"Test AUC score: {score}")

# Your Turn
Now you'll build your own **[baseline model](#$NEXT_NOTEBOOK_URL$)** which you can improve with feature engineering techniques as you go through the course.
